In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/Shared \drives/ProjectX \2021/data
!ls

Mounted at /content/drive
/content/drive/Shared drives/ProjectX 2021/data
 All_Malaria_Weather_TS.csv	  odisha_weather.gsheet
 All_Malaria_Weather_TS.gsheet	 'PROCESSED ODISHA MALARIA DATA'
 All_Malaria_Weather_TS_lit.csv  'RAW ODISHA MALARIA DATA'
 bangkok_dengue			  sarima_reg.ipynb
 dataframe.ipynb		 'Time Series Extraction.ipynb'
 odisha_weather.csv


In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import Embedding
from keras.layers import Masking
from keras.layers import SimpleRNN
from keras.layers import Activation
import keras.layers as kl
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import numpy as np
import os
import time
import sys
import sklearn as sk

In [3]:
weather_df = pd.read_csv("All_Malaria_Weather_TS.csv", index_col = 0)

In [4]:
weather_df.head()

,Year,Month,Region,Min_Temp,Max_Temp,Precipitation,Malaria_Cases
0,2003,1,Angul,13.6,27.1,2,805.0
1,2003,2,Angul,17.5,31.0,27,807.0
2,2003,3,Angul,20.6,34.9,47,1057.0
3,2003,4,Angul,26.3,40.0,6,1029.0
4,2003,5,Angul,28.5,41.4,14,707.0


In [5]:
weather_df.shape

(3960, 7)

In [6]:
cases = weather_df["Malaria_Cases"]

In [7]:
weather = weather_df.drop(columns = ["Malaria_Cases", "Region"])

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(weather, cases, test_size=.2, shuffle=False)

#sk.train_test_split(values, test_size=.2, shuffle=False)

In [9]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(3168, 5)
(3168,)
(792, 5)
(792,)


In [10]:
x_traint = x_train.transpose()
print(x_traint.shape)
x_traint2 = tf.expand_dims(x_traint, axis=0)
x_traint22 = np.array(x_traint2)
#print(x_traint22.shape)
#print(x_traint22)
y_traint = y_train.transpose()
print(y_traint.head())
y_traint2 = tf.expand_dims(y_traint, axis=0)
y_traint22 = np.array(y_traint2)
#print(y_traint22.shape)
#print(x_traint22)

(5, 3168)
0     805.0
1     807.0
2    1057.0
3    1029.0
4     707.0
Name: Malaria_Cases, dtype: float64


In [11]:
x_train2 = tf.expand_dims(x_train, axis=2)
x_train22 = np.array(x_train2)
print(len(x_train2))
print(x_train22.shape)
x_test2 = tf.expand_dims(x_test, axis=2)
x_test22 = np.array(x_test2)
print(len(x_test2))
print(x_test22.shape)
y_train2 = tf.expand_dims(y_train, axis=1)
y_train22 = np.array(y_train2)
print(len(y_train2))
print(y_train22.shape)
y_train3 = tf.expand_dims(y_train2, axis=2)
y_train33 = np.array(y_train3)
print(len(y_train3))
print(y_train33.shape)
xtry = tf.expand_dims(x_train, axis=0)
xtry2 = np.array(xtry)
print(xtry.shape)
ytry = tf.expand_dims(y_train2, axis=0)
ytry2 = np.array(ytry)
print(ytry.shape)
ytry3 = tf.expand_dims(y_train, axis=0)
ytry3 = np.array(ytry3)
print(ytry3.shape)
#y_train_trans = 

3168
(3168, 5, 1)
792
(792, 5, 1)
3168
(3168, 1)
3168
(3168, 1, 1)
(1, 3168, 5)
(1, 3168, 1)
(1, 3168)


In [12]:
x_train_np = np.array(x_train)
print(len(x_train))
print(x_train_np.shape)

y_train_np = np.array(y_train)
print(len(y_train))
print(y_train_np.shape)

3168
(3168, 5)
3168
(3168,)


In [36]:
from keras.metrics import FalseNegatives
model = Sequential()
#model.add(Embedding(input_dim=3186, output_dim=5))
model.add(LSTM(units = 1000, input_shape = (1,5), return_sequences=True))
model.add(LSTM(units = 1))
# model.add(Dropout(0.2))
#model.add(LSTM(4, return_sequences = True))
#model.add(Dropout(0.2))
#model.add(RepeatVector(1))
#model.add(Dense(y_train33.shape[0],activation='softmax'))
model.summary()
model.compile(loss='MSE', optimizer='adam', metrics=['mean_absolute_error', 'mse'])
#input_shape=(x_train.shape[0], x_train.shape[1])


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_32 (LSTM)              (None, 1, 1000)           4024000   
                                                                 
 lstm_33 (LSTM)              (None, 1)                 4008      
                                                                 
Total params: 4,028,008
Trainable params: 4,028,008
Non-trainable params: 0
_________________________________________________________________


In [ ]:
xxxx = x_train_np.reshape((3168,1,5))
yyyy = y_train22.reshape((3168))
yyyy /= yyyy.max()
model.fit(xxxx, yyyy, epochs=50, batch_size=64)
#validation_data = (x_test22, y_test), 

In [ ]:
import matplotlib.pyplot as plt
# plt.hist(xxxx[:,:,4], bins=50)
plt.hist(np.power(yyyy, 1/2.5), bins=50)

In [ ]:
model2 = Sequential()
#model2.add(kl.Conv1D(input_shape = (1, 3168, 5), filters =5, kernel_size = 5))
model2.add(Dense(16, input_dim=5, activation='relu'))
model2.add(Activation('relu'))
model2.add(Dense(1, activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
print(x_train.shape)
print(y_train.shape)

(3168, 5)
(3168,)


In [ ]:
model2.fit(x_train, y_train, epochs=20, batch_size=64)

Epoch 1/20
50/50 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - mean_absolute_error: 830.3885
Epoch 2/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - mean_absolute_error: 830.3885
Epoch 3/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - mean_absolute_error: 830.3885
Epoch 4/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - mean_absolute_error: 830.3885
Epoch 5/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - mean_absolute_error: 830.3885
Epoch 6/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - mean_absolute_error: 830.3885
Epoch 7/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - mean_absolute_error: 830.3885
Epoch 8/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - mean_absolute_error: 830.3885
Epoch 9/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0000e+